<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/Hw2Dolev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import random
import ipywidgets as widgets
import IPython.display as display
import os
import gdown
from firebase import firebase

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

# Function to retrieve all questions from Firebase
def get_all_questions():
    result = FBconn.get('/Questions', None)
    questions = []
    if result:
        for key, value in result.items():
            question = {
                'question': value['question'],
                'options': value['options'],
                'correctOption': value['correctOption']
            }
            questions.append(question)
    return questions

def download_image(url, filename):
    gdown.download(url, filename, quiet=False)

def quizScreen():
    # Create a submit button
    submit_button = widgets.Button(description='Submit', button_style='success')

    # Add a question and radio buttons
    def add_question(question, options):
        question_widget = widgets.HTML(value=f"<p style='font-size: 16px; color: #333;'><b>{question}</b></p>")
        options_widget = widgets.RadioButtons(options=options, layout={'display': 'flex', 'flex_flow': 'column'})
        options_widget.index = None  # Remove default selection
        return widgets.VBox([question_widget, options_widget])

    # Create a list to store question containers
    question_containers = []

    # Get all questions
    questions = get_all_questions()

    # Shuffle the questions randomly
    random.shuffle(questions)

    # Add questions to the containers
    for question in questions:
        question_widget = add_question(question['question'], question['options'])
        question_containers.append(question_widget)

    # Set the initial question index to 0
    current_question_index = 0

    # Create a burndown graph widget
    burndown_graph = widgets.Box(layout=widgets.Layout(display='flex', flex_flow='row', justify_content='center'))

    # Function to update the burndown graph
    def update_burndown_graph():
        burndown_icons = [widgets.Image(value=open('image.jpg', 'rb').read(), format='jpg', width=32, height=32)
                          for _ in range(current_question_index)]
        burndown_graph.children = burndown_icons

    # Update the burndown graph initially
    update_burndown_graph()

    # Function to handle button click event
    def submit_button_clicked(button):
        nonlocal current_question_index

        # Get selected option
        selected_option = question_containers[current_question_index].children[1].value

        if selected_option is None:
            return

        # Check if the selected option is correct
        correct_option = questions[current_question_index]['correctOption']
        is_correct = selected_option == correct_option

        # Move to the next question
        current_question_index += 1

        if current_question_index < len(question_containers):
            display.clear_output()
            display.display(burndown_graph)  # Display burndown graph at the top
            display.display(question_containers[current_question_index])
            display.display(submit_button)
        else:
            display.clear_output()
            print("Quiz completed!")

        # Update the burndown graph
        update_burndown_graph()

    # Set the style for the question containers
    question_style = "background-color: #f5f5f5; padding: 10px; margin-bottom: 10px; border-radius: 5px; box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.1);"

    # Display the burndown graph at the top
    burndown_graph_container = widgets.Box([burndown_graph], layout=widgets.Layout(justify_content='center'))
    display.display(burndown_graph_container)

    # Display the first question container
    question_container = widgets.Box([question_containers[current_question_index]], layout=widgets.Layout(justify_content='center', min_width='500px', width='50%', margin='0 auto', style=question_style))
    display.display(question_container)

    # Display the submit button
    submit_button_container = widgets.Box([submit_button], layout=widgets.Layout(justify_content='center'))
    display.display(submit_button_container)

    # Register the button click event handler
    submit_button.on_click(submit_button_clicked)

    # Download the image from Google Drive
    download_image('https://drive.google.com/uc?id=1qPBpXmgs0W-Y9919mj0WW29Hio8mikHv', 'image.jpg')

# Call the quizScreen function to start the quiz
quizScreen()


Quiz completed!


In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import random
import ipywidgets as widgets
import IPython.display as display
import time

def quizScreen():
    # Create a submit button
    submit_button = widgets.Button(description='Submit')

    # Create a question widget
    question_widget = widgets.HTML()

    # Create a radio button widget
    options_widget = widgets.RadioButtons()

    # Create a box to hold the question and options
    question_container = widgets.VBox([question_widget, options_widget])

    # Create a list to store burndown icons
    burndown_icons = []

    # Create a burndown graph widget
    burndown_graph = widgets.HTML()

    # Get all questions
    questions = get_all_questions()

    # Shuffle the questions randomly
    random.shuffle(questions)

    # Set the initial question index to 0
    current_question_index = 0

    # Create an array to store the time taken for each question
    time_array = [-1] * len(questions)

    # Create a variable to store the total time taken
    total_time = 0

    # Function to update the question and options
    def update_question():
        nonlocal current_question_index

        # Check if all questions have been answered
        if current_question_index >= len(questions):
            display.clear_output()
            statisticScreen(burndown_icons, time_array, total_time)
            return

        # Get the current question and options
        question = questions[current_question_index]['question']
        options = questions[current_question_index]['options']

        # Update the question widget and options widget
        question_widget.value = f"<p>{question}</p>"
        options_widget.options = options

        # Reset the options widget selection
        options_widget.value = None

        # Function to calculate and update the time taken for the current question
        def update_time(change):
            nonlocal total_time
            nonlocal current_question_index

            if current_question_index < len(questions) and options_widget.value is not None:
                current_time = time.time() - start_time
                time_array[current_question_index] = round(current_time, 2)

        # Register the update_time function to be called when the options are changed
        options_widget.observe(update_time, 'value')

        # Start the timer for the current question
        start_time = time.time()

    # Function to handle button click event
    def submit_button_clicked(button):
        nonlocal current_question_index

        # Check if an option has been selected
        if options_widget.value is None:
            return

        # Check if the selected option is correct
        selected_option = options_widget.value
        correct_option = questions[current_question_index]['correctOption']
        is_correct = selected_option == correct_option

        # Add 'V' or 'X' to burndown_icons based on correctness
        burndown_icons.append('V' if is_correct else 'X')

        # Increment the current_question_index
        current_question_index += 1

        # Update the burndown graph
        update_burndown_graph()

        # Update the question and options
        update_question()

    # Function to update the burndown graph
    def update_burndown_graph():
        graph_html = '<p style="font-size: 24px;">'
        for icon in burndown_icons:
            graph_html += f'<span style="color: green; font-weight: bold;">{icon}</span>' if icon == 'V' else f'<span style="color: red; font-weight: bold;">{icon}</span>'
        graph_html += '</p>'
        burndown_graph.value = graph_html

    # Update the question and options initially
    update_question()

    # Create a box to hold the burndown graph and submit button
    bottom_container = widgets.HBox([submit_button])

    # Create a box to hold the entire screen
    screen_container = widgets.VBox([burndown_graph, question_container, bottom_container])

    # Display the screen container
    display.display(screen_container)

    # Register the button click event handler
    submit_button.on_click(submit_button_clicked)

def statisticScreen(burndown_icons, time_array, total_time):
    # Create a widget to display the burndown graph
    burndown_widget = widgets.HTML()
    
    # Create a widget to display the time array
    time_widget = widgets.HTML()

    # Format the burndown icons as HTML
    burndown_html = '<p style="font-size: 24px;">'
    for icon in burndown_icons:
        burndown_html += f'{icon}'
    burndown_html += '</p>'
    burndown_widget.value = burndown_html

    # Format the time array as HTML
    time_html = '<p style="font-size: 24px;">'
    for time in time_array:
        time_html += f"{time}<br>"
    time_html += f"Total Time: {total_time}"
    time_html += '</p>'
    time_widget.value = time_html

    # Create a box to hold the statistics
    statistics_container = widgets.VBox([burndown_widget, time_widget])

    # Display the statistics container
    display.display(statistics_container)

# Start the quiz
quizScreen()
